# 네이버 환율 정보를 크롤링해서 환율을 주기적으로 알림받고 싶은 분들에게 텔레그램으로 알림을 주는 서비스입니다.
# 원하는 정보의 통화를 입력하고 환전 방식을 숫자로 입력하면 얼마를 환전받을 수 있는지 정보도 제공하도록 하였습니다.

# 라이브러리 설치

In [ ]:
!pip install schedule

In [ ]:
!pip install selenium

In [1]:
# HTTP 통신
import requests
from urllib.parse import urlencode, quote_plus, urlparse
from urllib.request import urlopen
import urllib

# 데이터 가공
from bs4 import BeautifulSoup
import json

#크롤링
import requests
from bs4 import BeautifulSoup
import pandas as pd

#selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

# 텔레그램 챗봇 연결

In [2]:
API_key_Telegram = "1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ"

In [2]:
def get_myChatID(parameter):
#     '''parameter = ~을 의미한다
#     이 코드셀에서 에러가 난다면, 해당 텔레그램 봇에서 직접 아무 메시지나 "두 번" 입력'''
    # 이 코드셀에서 에러가 난다면, 해당 텔레그램 봇에서 직접 아무 메시지나 "두 번" 입력
    endpoint = 'https://api.telegram.org/bot'
    query = '/' + 'getUpdates'
    URL = endpoint + API_key_Telegram + query
    
    print(URL)
    
    request = urllib.request.Request(URL)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    request_body = urlopen(request).read()

    if rescode == 200 :
        print(rescode)
        print('SUCCESS \n')
        chat_id = json.loads(request_body)['result'][0]['message']['from']['id']
        print('나의 chat ID : '+ str(chat_id))
     
        return str(chat_id)
    
    else :
        print(rescode)
        print('FAILED')

In [4]:
# 이 함수가 에러 없이 실행되면 텔레그램에 무사히 연동된 것이다.
get_myChatID()

https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034


'680310034'

In [5]:
def send_message(input_text) :
    
    # 유저마다 chat id가 다르다.
    chat_id = get_myChatID()
    
    endpoint = 'https://api.telegram.org/bot'
    action = '/' + 'sendmessage'
    param_list = [
        'chat_id=' + str(chat_id), 
        'text=' + input_text
    ]
    param = '&'.join(param_list)
    query = action + '?' + param
    
    URL = endpoint + API_key_Telegram + query
    print(URL)
    
    try : 
        r = requests.get(URL)
        print(r.text)
    
    except Exception as e:
        print(str(e))

In [6]:
def read_message():
    endpoint = 'https://api.telegram.org/bot'
    query = '/' + 'getUpdates'
    URL = endpoint + API_key_Telegram + query
    #print(URL)
    
    request = urllib.request.Request(URL)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    request_body = urlopen(request).read()
    
    if rescode == 200:
        text = json.loads(request_body)['result'][-1]['message']['text']
        return text

# 네이버 환율사이트 크롤링

In [7]:
def currency_crawler():
    url = "https://finance.naver.com/marketindex/"
    req = requests.get(url).text
    soup = BeautifulSoup(req, "html.parser")
    second = soup.find("div",{"class":"selectbox-noscript"})
    lists = second.find_all("option")
    Cdict = {}
    for item in lists:
        threeDetails = item.text
        key = threeDetails.split(" ")[-3]
        unit = threeDetails.split(" ")[-2]
        symbol = threeDetails.split(" ")[-1]
        value = []
        value.append(item["value"])
        value.append(unit)
        value.append(symbol)
        Cdict[key] = value
    return Cdict 

In [8]:
currency_crawler()

{'대한민국': ['1', '원', 'KRW'],
 '미국': ['1204.9', '달러', 'USD'],
 '유럽연합': ['1300.39', '유로', 'EUR'],
 '일본': ['10.7513', '엔', 'JPY'],
 '중국': ['171.11', '위안', 'CNY'],
 '홍콩': ['154.73', '달러', 'HKD'],
 '대만': ['39.68', '달러', 'TWD'],
 '영국': ['1552.45', '파운드', 'GBP'],
 '오만': ['3129.37', '리알', 'OMR'],
 '캐나다': ['909.12', '달러', 'CAD'],
 '스위스': ['1225.11', '프랑', 'CHF'],
 '스웨덴': ['122.87', '크로나', 'SEK'],
 '호주': ['797.34', '달러', 'AUD'],
 '뉴질랜드': ['762.88', '달러', 'NZD'],
 '체코': ['51.9', '코루나', 'CZK'],
 '칠레': ['1.49', '페소', 'CLP'],
 '터키': ['197.32', '리라', 'TRY'],
 '몽골': ['0.44', '투그릭', 'MNT'],
 '이스라엘': ['351.65', '세켈', 'ILS'],
 '덴마크': ['174.14', '크로네', 'DKK'],
 '노르웨이': ['129.31', '크로네', 'NOK'],
 '사우디아라비아': ['321.24', '리얄', 'SAR'],
 '쿠웨이트': ['3938.87', '디나르', 'KWD'],
 '바레인': ['3196.11', '디나르', 'BHD'],
 '아랍에미리트': ['328.03', '디르함', 'AED'],
 '요르단': ['1699.44', '디나르', 'JOD'],
 '이집트': ['77.44', '파운드', 'EGP'],
 '태국': ['38.16', '바트', 'THB'],
 '싱가포르': ['860.52', '달러', 'SGD'],
 '말레이시아': ['287.57', '링깃', 'MYR'],
 '인도

In [9]:
Cdict = currency_crawler()

# 실행 코드

## 챗봇 대화(?)

In [10]:
send_message("환율을 알고 싶은 나라를 입력해주세요!")

https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/sendmessage?chat_id=680310034&text=환율을 알고 싶은 나라를 입력해주세요!
{"ok":true,"result":{"message_id":33,"from":{"id":1061348148,"is_bot":true,"first_name":"Currency_bot","username":"AhkyoungBot"},"chat":{"id":680310034,"first_name":"\uc5f0\uacbd","last_name":"\ud64d","type":"private"},"date":1582249783,"text":"\ud658\uc728\uc744 \uc54c\uace0 \uc2f6\uc740 \ub098\ub77c\ub97c \uc785\ub825\ud574\uc8fc\uc138\uc694!"}}


In [12]:
want = read_message()
want

'대만'

In [ ]:
type(want)

In [13]:
send_message(want + " "+Cdict[want][1] +"의 환율은 "+ Cdict[want][0] + " 원 입니다.")

https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/sendmessage?chat_id=680310034&text=대만 달러의 환율은 39.68 원 입니다.
{"ok":true,"result":{"message_id":35,"from":{"id":1061348148,"is_bot":true,"first_name":"Currency_bot","username":"AhkyoungBot"},"chat":{"id":680310034,"first_name":"\uc5f0\uacbd","last_name":"\ud64d","type":"private"},"date":1582249802,"text":"\ub300\ub9cc \ub2ec\ub7ec\uc758 \ud658\uc728\uc740 39.68 \uc6d0 \uc785\ub2c8\ub2e4."}}


In [14]:
send_message("원화를 "+want +" "+ Cdict[want][1]+" 로 바꾸고 싶다면 1\n"+want+" "+Cdict[want][1]+"를 원화로 바꾸고 싶다면 2를 눌러주세요")

https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/sendmessage?chat_id=680310034&text=원화를 대만 달러 로 바꾸고 싶다면 1
대만 달러를 원화로 바꾸고 싶다면 2를 눌러주세요
{"ok":true,"result":{"message_id":36,"from":{"id":1061348148,"is_bot":true,"first_name":"Currency_bot","username":"AhkyoungBot"},"chat":{"id":680310034,"first_name":"\uc5f0\uacbd","last_name":"\ud64d","type":"private"},"date":1582249808,"text":"\uc6d0\ud654\ub97c \ub300\ub9cc \ub2ec\ub7ec \ub85c \ubc14\uafb8\uace0 \uc2f6\ub2e4\uba74 1\n\ub300\ub9cc \ub2ec\ub7ec\ub97c \uc6d0\ud654\ub85c \ubc14\uafb8\uace0 \uc2f6\ub2e4\uba74 2\ub97c \ub20c\ub7ec\uc8fc\uc138\uc694"}}


In [15]:
how = read_message()
try:
    if int(how)==1:
        whichway = 1 
    elif int(how)==2:
        whichway = 2
except:
    send_message("1 또는 2를 입력해주세요!")

In [16]:
send_message("얼마를 환전하실건가요?")

https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/sendmessage?chat_id=680310034&text=얼마를 환전하실건가요?
{"ok":true,"result":{"message_id":38,"from":{"id":1061348148,"is_bot":true,"first_name":"Currency_bot","username":"AhkyoungBot"},"chat":{"id":680310034,"first_name":"\uc5f0\uacbd","last_name":"\ud64d","type":"private"},"date":1582250000,"text":"\uc5bc\ub9c8\ub97c \ud658\uc804\ud558\uc2e4\uac74\uac00\uc694?"}}


In [17]:
amount = read_message()
amount

'1000달러'

In [18]:
num = amount[0:-2]

In [19]:
if whichway == 1:
    won = int(num) * 10000
    total = round(won/ float(Cdict[want]),2)
    send_message(str(total)+ " "+want+" "+Cdict[want][1]+"를 환전받게 되실겁니다.")
elif whichway == 2:
    dollar = int(num)
    total = round(dollar * float(Cdict[want][0]),2)
    send_message(str(total) + "원을 환전받게 되실겁니다")
    


https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/sendmessage?chat_id=680310034&text=39680.0원을 환전받게 되실겁니다
{"ok":true,"result":{"message_id":40,"from":{"id":1061348148,"is_bot":true,"first_name":"Currency_bot","username":"AhkyoungBot"},"chat":{"id":680310034,"first_name":"\uc5f0\uacbd","last_name":"\ud64d","type":"private"},"date":1582250031,"text":"39680.0\uc6d0\uc744 \ud658\uc804\ubc1b\uac8c \ub418\uc2e4\uac81\ub2c8\ub2e4"}}


In [ ]:
won = int(num)*10000
won

In [ ]:
import math

In [ ]:
total = round(won/ float(Cdict[want]),2)
total

In [ ]:
send_message(str(total)+ " "+want+" "+Cdict[want][1]+"를 환전받게 되실겁니다.")

# 하루에 4번 원하는 통화 환율 텔레그램으로 알림해주기

In [20]:
# 시간 관련 
import datetime
from time import sleep
import schedule

In [21]:
def job():
    send_message(want + " 달러의 환율은 "+ Cdict[want][0] + "원 입니다.")

In [22]:
schedule.every(1).minutes.do(job)

Every 1 minute do job() (last run: [never], next run: 2020-02-21 01:55:23)

In [ ]:
#6시간에 한번씩 알림 보내기
schedule.every(6).hour.do(job)

In [23]:
while True:
    schedule.run_pending()
    time.sleep(1)

https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/sendmessage?chat_id=680310034&text=대만 달러의 환율은 39.68원 입니다.
{"ok":true,"result":{"message_id":41,"from":{"id":1061348148,"is_bot":true,"first_name":"Currency_bot","username":"AhkyoungBot"},"chat":{"id":680310034,"first_name":"\uc5f0\uacbd","last_name":"\ud64d","type":"private"},"date":1582250127,"text":"\ub300\ub9cc \ub2ec\ub7ec\uc758 \ud658\uc728\uc740 39.68\uc6d0 \uc785\ub2c8\ub2e4."}}
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/getUpdates
200
SUCCESS 

나의 chat ID : 680310034
https://api.telegram.org/bot1061348148:AAF_BxHrEBzRQ-RkPIUCALIOC2CcFRoSQgQ/sendmessage?chat_id=680310034&text=대만 달러의 환율은 39.68원 입니다.
{"ok":true,"result":{"message_id":42,"from":{"id":1061348148,"is_bot":true,"first_name":"Currency_bot","username":"AhkyoungBot"},"chat":{"id":680310034,"firs

KeyboardInterrupt: 